# 🚆Train
지하철 데이터 전처리

### TODO

1. 지하철 정류장 데이터 
   - [X] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
   - [ ] 결측값 확인
     - [X] 위경도 결측값 분석
     - [X] 결측값 채워진 데이터셋 구하기
     - [X] 노선명 통일
     - [ ] 역사명 통일
     - [ ] 두 데이터셋 하나로 결합
     - [ ] 각 노선별 검토 
   - [ ] 서울 및 각 역 위치 지도에 시각화
   - [ ] 이상값 확인
   - [ ] 수도권 바깥 데이터 삭제 
   - [ ] 필요없는 로우 및 컬럼 삭제 
   - [ ] 환승역, 종점, 분기역 정보 가공
2. 지하철 시간표 데이터
   - [ ] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
   - [ ] 결측값 및 이상값 확인
   - [ ] 방향별 종점별 막차 시간표 정보로 가공

### 0. import

In [144]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import trange, notebook

### 1. 지하철 정류장 데이터

##### 1-a. 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  

In [145]:
# 데이터 읽어오기
station = pd.read_csv('train_station.csv')

# 확실히 필요없는 컬럼 제거
station = station.drop(columns=['영문역사명', '한자역사명', '운영기관명', '역사도로명주소', '역사전화번호','데이터기준일자'])

# 확실히 필요없는 행 제거
# # 인천 1~2호선
# station = station[(station['노선번호'].str.slice(stop=3) != 'S28')]
# 인천 자기부상철도 -> 휴업 상태
station = station[(station['노선번호'] != 'S28M1')]
# 대전
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '30')]
# 대구 1~3호선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '27')]
# 부산 1~4호선, 김해선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '26') & (station['노선번호'] != 'L48B1')]
# 광주
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '29')]

# # 나중을 위해 역번호 저장
# station_id = station['역번호']

station = station.reset_index(drop=True)
station

,역번호,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역위도,역경도
0,3110,계양,S2801,인천지하철 1호선,환승역,S2801+I28A1,공항철도,37.571539,126.736319
1,3111,귤현,S2801,인천지하철 1호선,일반역,-,-,37.566362,126.742498
2,3112,박촌,S2801,인천지하철 1호선,일반역,-,-,37.553525,126.744946
3,3113,임학,S2801,인천지하철 1호선,일반역,-,-,37.545058,126.738642
4,3114,계산,S2801,인천지하철 1호선,일반역,-,-,37.543243,126.728436
...,...,...,...,...,...,...,...,...,...
748,4811,달미역,I41WS,서해선,일반역,NaN,NaN,37.349320,126.808218
749,4812,선부역,I41WS,서해선,일반역,NaN,NaN,37.333908,126.810977
750,4813,초지역,I41WS,서해선,환승역,"I4103, I28K1","수인선, 과천안산선",37.320583,126.806151
751,4814,시우역,I41WS,서해선,일반역,NaN,NaN,37.313767,126.798303


##### 1-b. 결측값 확인

1-b-i. 위경도 결측값 분석

In [146]:
# 역이 노선별로 총 몇 개인지 확인
# 1호선이어도, 구간별로 노선 명칭 다름 (우리가 아는 1호선 = 경원선 + 1호선 + 경인선 + 경부선 + 장항선)
total_station_cnt_by_route = station["노선명"].value_counts().sort_index()
total_station_cnt_by_route

1호선               10
2호선               50
3호선               34
4호선               26
5호선               56
6호선               39
7호선               42
8호선               18
경강선               11
경부선               45
경원선               24
경의중앙선             57
경인선               20
경춘선               24
김포골드라인            10
도시철도 7호선          11
분당선               37
서해선               12
수도권  도시철도 9호선     25
수도권 경량도시철도 신림선    11
수인선               26
신분당선              16
안산과천선             21
에버라인              15
우이신설선             13
의정부               15
인천국제공항선           14
인천지하철 1호선         30
인천지하철 2호선         27
일산선               11
진접선                3
Name: 노선명, dtype: int64

In [147]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
null_x_station_cnt = station['노선명'][station['역경도'].isnull(
)].value_counts().sum()
null_y_station_cnt = station['노선명'][station['역위도'].isnull(
)].value_counts().sum()
print("위도 결측값 개수 :", null_y_station_cnt)
print("경도 결측값 개수 :", null_x_station_cnt)

# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
null_station_cnt_by_route = station['노선명'][station['역위도'].isnull(
)].value_counts().sort_index()
null_station_cnt_by_route

위도 결측값 개수 : 352
경도 결측값 개수 : 352


1호선              10
2호선              50
3호선              34
4호선              26
5호선              56
6호선              39
7호선              42
8호선              18
김포골드라인           10
도시철도 7호선         11
수도권  도시철도 9호선    25
우이신설선            13
의정부              15
진접선               3
Name: 노선명, dtype: int64

1-b-ii. 결측값 채워진 데이터셋 구하기

In [148]:
# 1. 노선별 전체 역 개수가 실제 역 개수와 다름
# 2. 위경도 결측 데이터가 존재하는 노선에 대해 알아본 결과, 서울시 역사마스터 정보와 제대로 합처지지 않은 것으로 보임
# -> 서울시 역사마스터 정보와의 join 필요

# 서울시 역사마스터 데이터 읽어오기
station_seoul = pd.read_csv('train_station_seoul.csv', encoding='euc_kr')

# 기존 데이터셋과 컬럼 이름 맞추기
station_seoul.rename(columns={'역사_ID': '역번호', '호선': '노선명'}, inplace=True)
station.rename(columns={'역위도': '위도', '역경도': '경도'}, inplace=True)

# 서울시 역사마스터 데이터, 위도와 경도가 반대로 들어가 있는 문제 해결
station_seoul.rename(columns={'위도': '경도', '경도': '위도'}, inplace=True)

# 필요없는 노선 제거
station_seoul = station_seoul[(station_seoul['노선명'] != '장항선')]

station_seoul

,역번호,역사명,노선명,경도,위도
0,9996,미사,5호선,127.193877,37.560927
1,9995,강일,5호선,127.175930,37.557490
2,4929,김포공항,김포골드라인,126.801868,37.562360
3,4928,고촌,김포골드라인,126.770345,37.601243
4,4927,풍무,김포골드라인,126.732387,37.612488
...,...,...,...,...,...
760,154,종로5가,1호선,127.001849,37.570926
761,153,종로3가,1호선,126.991847,37.570406
762,152,종각,1호선,126.982923,37.570161
763,151,시청,1호선,126.977088,37.565715


In [149]:
# 역이 노선별로 몇 개인지 확인
total_station_seoul_cnt_by_route = station_seoul["노선명"].value_counts().sort_index()
total_station_seoul_cnt_by_route

# 확인 결과, 앞서 전체 역정보 데이터셋에 빠진 일부 역이 이 데이터셋에는 포함되어 있음 (역 개수 더 많음)

1호선          10
2호선          50
3호선          34
4호선          26
5호선          58
6호선          39
7호선          53
7호선(인천)       9
8호선          18
9호선          25
9호선(연장)      13
경강선          11
경부선          39
경원선          30
경의중앙선        31
경인선          20
경춘선          19
공항철도1호선      14
과천선           9
김포골드라인       10
분당선          35
서해선          12
수인선          18
신림선          11
신분당선          6
신분당선(연장)      7
신분당선(연장2)     3
안산선          13
에버라인선        15
우이신설선        13
의정부선         15
인천1호선        30
인천2호선        27
일산선          11
중앙선          21
진접선           3
Name: 노선명, dtype: int64

In [150]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
null_x_station_seoul_cnt = station_seoul['노선명'][station_seoul['경도'].isnull(
)].value_counts().sum()
null_y_station_seoul_cnt = station_seoul['노선명'][station_seoul['위도'].isnull(
)].value_counts().sum()
print("위도 결측값 개수 :", null_y_station_seoul_cnt)
print("경도 결측값 개수 :", null_x_station_seoul_cnt)

# 확인 결과, 해당 데이터셋은 위경도 정보가 빠진 것 없이 잘 들어있음 

위도 결측값 개수 : 0
경도 결측값 개수 : 0


1-b-iii. 노선명 통일

In [151]:
# 둘 중 한 데이터셋에만 있는 노선명 확인
route_name = set(total_station_cnt_by_route.index)
route_name_seoul = set(total_station_seoul_cnt_by_route.index)
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

{'안산과천선', '인천지하철 1호선', '도시철도 7호선', '에버라인', '인천국제공항선', '수도권 경량도시철도 신림선', '인천지하철 2호선', '의정부', '수도권  도시철도 9호선'}
{'인천2호선', '의정부선', '공항철도1호선', '중앙선', '과천선', '9호선(연장)', '에버라인선', '인천1호선', '안산선', '신림선', '신분당선(연장2)', '7호선(인천)', '신분당선(연장)', '9호선'}


In [152]:
# 노선명 통일
station = station.replace({"도시철도 7호선": "7호선"})
station = station.replace({"수도권  도시철도 9호선": "9호선"})
station = station.replace({"수도권 경량도시철도 신림선": "신림선"})
station = station.replace({"인천지하철 1호선": "인천1호선"})
station = station.replace({"인천지하철 2호선": "인천2호선"})
station = station.replace({"인천국제공항선": "공항철도1호선"})
station = station.replace({"에버라인": "에버라인선"})
station = station.replace({"의정부": "의정부선"})

# 둘 중 한 데이터셋에만 있는 노선명 재확인
route_name = set(station["노선명"].value_counts().index)
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

# 안산과천선의 경우, 역사명 통일할 때 안산선 / 과천선으로 나눌 예정

{'안산과천선'}
{'중앙선', '과천선', '9호선(연장)', '안산선', '7호선(인천)', '신분당선(연장2)', '신분당선(연장)'}
